# Import Libraries

In [2]:
import pandas as pd
import numpy as np
import cPickle as pickle
import unidecode

with open('master_total_df.p','rb') as f:
    master_total_df = pickle.load(f)
    
print master_total_df.shape

(22707, 16)


In [3]:
master_total_df.head(3)

,index,company,jobdesc,city,state,title,sourcesite,views,days_posted,post_start_date,link,base_title,parsed_title,parsed_title_i,expanded_title,prefix_title
0,0,Advocate Health Care,"\nAs part of Advocate Health Care, Advocate Ch...",Oak Lawn,IL,Clinical Practice Specialist - 4 Hope,ind,0,NaN,NaN,http://www.indeed.com/viewjob?jk=6244f7f3a4861...,specialist,"[clinical, practice, specialist, , , 4, hope]",[2],practice specialist,practice
1,1,University of Washington Medical Center,\nThe University of Washington (UW) is proud t...,Seattle,WA,WEB DEVELOPER,ind,0,NaN,NaN,http://www.indeed.com/viewjob?jk=3d6a9f18f5301...,developer,"[web, developer]",[1],web developer,web
2,2,Macy's,"\nJob Overview:\n\n\nThe Analyst, Marketing An...",New York,NY,"Analyst, Marketing Analytics",ind,0,NaN,NaN,http://www.indeed.com/viewjob?jk=d94a7d13dfbb0...,analyst,"[analyst, , marketing, analytics]",[0],analyst,analyst


In [4]:
jobdesc = master_total_df[['link','jobdesc']].copy()

def uncode(x):
    try:
        return unidecode.unidecode(x.decode('utf-8')).replace('\n',' -').lower()

    except:
        try:
            return unidecode.unidecode(x).replace('\n',' -').lower()
        except:
            print x
            return x 
jobdesc['cleandesc'] = jobdesc['jobdesc'].map(uncode)

# Split on /n - then only take single sentence clusters

# Sample Job description - identify key lists

Note that the items have the following features:
- usually single sentence
- not multi-paragraph
- usually have a new line
- sections are separated by double returns

In [5]:
print jobdesc['jobdesc'][17]


The Application Developer II provides technical programming expertise to members of the Enterprise Applications Sector in support of MIT Lincoln Laboratory SAP (SRM, XI, BW, ECC) and legacy systems. The Application Developer works with Business Analyst functional system design specifications to create technical specifications that are then programmed and developed in accordance with development standards.


Technical Support

With guidance from Development Team Lead and senior developers, provides technical support in the form of analysis, coding, testing, transport coordination and documenting all work related to SAP, the Internet and legacy systems where applicable.
Provides technical support in various application areas including working with technical developers in Project Systems as well as users in the Workforce Service Center and developers at MIT campus.
Works to learn proper standards and efficient techniques for coding, analysis and documentation.
Provides support in the BI 

# How to split items from descriptions

1. will split by newlines
2. will count how many sentences by periods
3. will only take single sentences
4. due to the "split" string approach, if the 2nd part of the string is empty, this will still qualify

Examples:
- "Foo is bar" will give one phrase back (True!)
- "Foo is bar. Bar is foo" will give two items (false!)
- "Foo is bar." will give two elements back, but 2nd phrase will be zero, so this will need to be captured as well. 

We will prototype with a sample job listing.

*Not all job postings will be the same, but we are hoping that the majority of job postings will follow this "list format" approach to isolate requirements

In [6]:
# prototype
for x in jobdesc['jobdesc'][12].split('\n'):
    print '==============='
    ct = len(x.strip().split('.'))
    if ct ==1:
        print '============>',x
    elif len(x.strip().split('.')[1])<=1:
        print '============>',x
    else:
        print x

============> 
============> Qualifications
============> Undergraduate or advanced degree in a quantitative discipline (i.e. Statistics, Mathematics, Econometrics, Operations Research)
============> Strong academic qualifications, including advanced understanding/coursework in database management and math (Linear, Algebra, Calculus)
============> 2-9 years of hands-on experience developing and applying predictive models and other advanced statistical approaches in a corporate or consulting setting, preferably in a marketing and sales context
============> Proficiency in statistical data analysis and data mining packages (e.g., R, SAS, SPSS, Alteryx, MatLab, STATA, Excel)
============> Advanced knowledge of data management tools including SQL/RDBMS, NoSQL (e.g. MongoDB), Hadoop and/or other big data technologies
============> Advanced programming skills in at least one of Java, Python, R, C++, C#, etc.
============> Experience linking multiple data platforms (social media, open, etc) a

In [7]:
def pullLI(y):
    LI = []
    y  = y.replace('i.e.','-ie-').replace('e.g.','-eg-')
    for x in y.split('\n'):
        ct = len(x.strip().split('.'))
        if ct ==1:
            if len(x)>0:
                LI.append(x)
        elif len(x.strip().split('.')[1])<=1:
            if len(x.strip().split('.')[0])>0:
                LI.append(x.strip().split('.')[0])
    return LI
jobdesc['LI'] = jobdesc['jobdesc'].map(pullLI)

def cleanLI(yy):
    newlist = []
    for y in yy:
        try:
            newlist.append(unidecode.unidecode(y.decode('utf-8')).lower())
        except:
            try:
                newlist.append(y.decode('utf-8').lower())
            except:
                newlist.append(y.lower())
    return newlist
jobdesc['LI'] = jobdesc['LI'].map(cleanLI)
jobdesc['LI_length'] = jobdesc['LI'].map(lambda x : len(x))

In [8]:
jobdesc.head()

,link,jobdesc,cleandesc,LI,LI_length
0,http://www.indeed.com/viewjob?jk=6244f7f3a4861...,"\nAs part of Advocate Health Care, Advocate Ch...","-as part of advocate health care, advocate ch...",[3+ years recent experience in clinical specia...,29
1,http://www.indeed.com/viewjob?jk=3d6a9f18f5301...,\nThe University of Washington (UW) is proud t...,-the university of washington (uw) is proud t...,"[enhance production systems:, * clearly define...",46
2,http://www.indeed.com/viewjob?jk=d94a7d13dfbb0...,"\nJob Overview:\n\n\nThe Analyst, Marketing An...","-job overview: - - -the analyst, marketing an...","[job overview:, essential functions:, % of tim...",24
3,http://www.indeed.com/viewjob?jk=c97751deb9e89...,\nSkyport Systems is developing groundbreaking...,-skyport systems is developing groundbreaking...,"[qualifications, b, experience in any of these...",17
4,http://www.indeed.com/viewjob?jk=fe4ef48c8fd03...,"\nCommand Alkon, a global leader in integrated...","-command alkon, a global leader in integrated...","[command alkon, a global leader in integrated ...",32


In [9]:
def split(y,phrase):
    try:
        if phrase in ('to','in'):
            return y.split(' '+phrase+' ')[1]
        else:
            return y.split(phrase)[1]
    except:
        return ''
    
jobdesc['in'] =jobdesc['LI'].map(lambda x : [split(y,'in') for y in x if split(y,'in')!=''] )
jobdesc['including'] =jobdesc['LI'].map(lambda x : [split(y.lower(),'including') for y in x if split(y.lower(),'including')!=''] )
jobdesc['knowledge of'] =jobdesc['LI'].map(lambda x : [split(y.lower(),'knowledge of') for y in x if split(y.lower(),'knowledge of')!=''] )
jobdesc['experience with'] =jobdesc['LI'].map(lambda x : [split(y.lower(),'experience with') for y in x if split(y.lower(),'experience with')!=''] )
jobdesc['understanding of'] =jobdesc['LI'].map(lambda x : [split(y.lower(),'understanding of') for y in x if split(y.lower(),'understanding of')!=''] )
jobdesc['to'] =jobdesc['LI'].map(lambda x : [split(y.lower(),'to') for y in x if split(y.lower(),'to')!=''] )


In [10]:
i = 67
print jobdesc['jobdesc'][i]
jobdesc['LI'][i]


General Overview:

We are seeking a highly motivated individual to take a lead role in the development of a Data Analytics & Reporting program to support the Spaulding Rehab Network. The ideal candidate will work directly with IT and our business partners to understand current data challenges, eliciting requirements to define, scope, and deliver solutions that meet the business intelligence needs. The candidate will be responsible for architecting, designing, developing and supporting reporting solutions. S/he will help us navigate the transition to Epic by providing guidance and expertise in the area of Epic Reporting solutions.

Responsibilities:

• Design and build queries, reports, and dashboards using business intelligence tools adhering to established frameworks and standards

• Develop migration strategies and processes to move data stored in disparate sources to SQL Server platform

• Actively participate in and influence discussions around reporting requirements, infrastructu

['general overview:',
 'responsibilities:',
 '* design and build queries, reports, and dashboards using business intelligence tools adhering to established frameworks and standards',
 '* develop migration strategies and processes to move data stored in disparate sources to sql server platform',
 '* actively participate in and influence discussions around reporting requirements, infrastructure needs, and design',
 '* interact directly with clarity report writers and clinical/business analysts to assure necessary data is available and accurate to support report writing and the use of analytics tools',
 '* consistently deliver high-quality business intelligence solutions and services to clients on schedule',
 '* bring structure to ambiguous business problems and recommend solutions that can be easily interpreted by users',
 '* serve as end user support for the organizational user base',
 '* follow, maintain and improve all documentation and procedures for report design and production',
 '

In [11]:
jobdesc['in'][i]

['disparate sources to sql server platform',
 'and influence discussions around reporting requirements, infrastructure needs, and design',
 'a dynamic business environment, with demonstrated reporting, analytical, and database experience; bs/ba required',
 'a healthcare environment, including use and experience with an ehr system',
 'a dynamic, collaborative, team environment']

In [12]:

jobdesc['including'][i]

[' use and experience with an ehr system',
 ' clarity, crystal reports, epic reporting workbench']

In [13]:

jobdesc['knowledge of'][i]

[]

In [14]:
jobdesc['experience with'][i]

[' an ehr system',
 ' relational structures, structured query language (sql), data warehouse and reporting techniques']

In [15]:
jobdesc['understanding of'][i]

[' bi best practices/ methodologies; experience with relational structures, structured query language (sql), data warehouse and reporting techniques']

In [16]:
jobdesc['to'][i]

['established frameworks and standards',
 'move data stored in disparate sources',
 'assure necessary data is available and accurate',
 'clients on schedule',
 'ambiguous business problems and recommend solutions that can be easily interpreted by users',
 'work in a dynamic, collaborative, team environment',
 'gather, synthesize, and organize data',
 'work collaboratively and effectively with business and systems personnel']

In [17]:
jobdesc[jobdesc['LI_length']>10].count()

link                14246
jobdesc             14246
cleandesc           14246
LI                  14246
LI_length           14246
in                  14246
including           14246
knowledge of        14246
experience with     14246
understanding of    14246
to                  14246
dtype: int64